In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
row=64
col=64
mydata = pd.read_csv(r"D:/athira/Courses/MS WPI/Fall'19/ML/Project_Distracted Driver Detection/Code/2 NN/train_gray_64x64.csv",header = None)


In [3]:
Input = mydata.iloc[0:,:-1].values/255.0
Label = mydata.iloc[0:,-1].values
for i in range(len(Label)):
    Label[i] = int(Label[i][1])

In [4]:
X=Input.copy()
Y=Label.copy()

In [5]:
X = np.array(list(x for x in X))
Y = np.array(list(x for x in Y))

In [6]:
print(Y[0:5])
Y=Y.T
print(Y.shape)
print(X.shape)
print(X[0])
print(X[0:3])
type(X), type(Y)

[0 0 0 0 0]
(22424,)
(22424, 4096)
[0.17254902 0.16862745 0.17647059 ... 0.04313725 0.46666667 0.05098039]
[[0.17254902 0.16862745 0.17647059 ... 0.04313725 0.46666667 0.05098039]
 [0.10196078 0.11764706 0.12156863 ... 0.04313725 0.03137255 0.03137255]
 [0.16470588 0.19607843 0.2        ... 0.08627451 0.08235294 0.10980392]]


(numpy.ndarray, numpy.ndarray)

In [7]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2,random_state = 50)

# X_train = X_train.reshape(X_train.shape[0],row,col)
# X_test = X_test.reshape(X_test.shape[0],row,col)
# print(X_train.shape)
# print(X_test.shape)

#class_names = ['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9']
class_names = [0,1,2,3,4,5,6,7,8,9]

In [8]:
print(Y_train[0:5])
type(X_train), type(Y_train), type(X_test), type(Y_test)

[0 4 0 7 7]


(numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray)

In [9]:
model = keras.Sequential([keras.layers.Flatten(),
                          keras.layers.Dense(256, activation=tf.nn.relu),
                          keras.layers.Dense(128, activation=tf.nn.relu),
                          keras.layers.Dense(10, activation=tf.nn.softmax)])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [10]:
model.fit(X_train,Y_train,epochs=10)

Epoch 1/10
17939/17939 [==============================] - 23s 1ms/step - loss: 1.2769 - acc: 0.5702
Epoch 2/10
17939/17939 [==============================] - 19s 1ms/step - loss: 0.3523 - acc: 0.9001
Epoch 3/10
17939/17939 [==============================] - 19s 1ms/step - loss: 0.1937 - acc: 0.9434
Epoch 4/10
17939/17939 [==============================] - 19s 1ms/step - loss: 0.1415 - acc: 0.9561
Epoch 5/10
17939/17939 [==============================] - 19s 1ms/step - loss: 0.1055 - acc: 0.9682
Epoch 6/10
17939/17939 [==============================] - 22s 1ms/step - loss: 0.0958 - acc: 0.9705
Epoch 7/10
17939/17939 [==============================] - 19s 1ms/step - loss: 0.0759 - acc: 0.9762: 4s - loss: 0.0577 - acc - ETA: 3s - loss: 0.0596 - acc: 0.98 - ETA: 3s - loss: 0.0598 - ETA: 2s - loss: 0.07 - ETA: 1s - loss
Epoch 8/10
17939/17939 [==============================] - 19s 1ms/step - loss: 0.0662 - acc: 0.9771
Epoch 9/10
17939/17939 [==============================] - 18s 1ms/step - 

In [11]:
test_loss,test_acc = model.evaluate(X_test,Y_test)
print('Split Test accuracy: ',test_acc)
predictions = model.predict(X_test)
print(predictions[0])
print(np.argmax(predictions[0]), ':',Y_test[0])

4485/4485 [==============================] - 1s 170us/step
Split Test accuracy:  0.9562987737033678
[5.3638212e-07 9.2711758e-09 4.5054414e-09 5.7342178e-12 1.9262947e-07
 6.2749600e-06 1.8760378e-06 1.5507429e-10 1.0744471e-03 9.9891663e-01]
9 : 9


In [12]:
from sklearn import metrics
import time
import cv2
import glob

In [ ]:
def matrix(row,col,imgs):
    a = row*col
    vector_newX = np.zeros((a, 1))
    imgSeq = []
    for img in imgs :
        image = cv2.imread(img,cv2.IMREAD_GRAYSCALE)/255
        imgSeq.append(img.split('img_')[1].split('.jpg')[0])
        img0 = cv2.resize(image,(row,col))
        flat = img0.reshape(a,1)
        vector_newX = np.c_[vector_newX,flat]
        print(img)
    vector_newX = vector_newX.T
    finalX_train = vector_newX[1:,:]
    print('size of feature martix is:',np.shape(finalX_train))
    return  finalX_train,imgSeq
testimgs = glob.glob(r"D:/athira/Courses/MS WPI/Fall'19/ML/Project_Distracted Driver Detection/Data/imgs/test 500/*.jpg")
combined_train,imgSeq = matrix(row,col,testimgs)

In [14]:
General_prediction = model.predict(combined_train)
Y_pred_case = np.empty(General_prediction.shape[0],dtype=object)
print(len(General_prediction))
for i in range(len(General_prediction)):
    l = np.argmax(General_prediction[i])
    Y_pred_case[i] = 'c'+ str(l)
print(Y_pred_case)
imgLabel = pd.read_csv("D:/athira/Courses/MS WPI/Fall'19/ML/Project_Distracted Driver Detection/Data/imgs/test 500/ImageLabels.csv")
imgLabel["NeuralNet_Gray"]=''
print(imgLabel.head(10))
correct=0
for i,val in enumerate(imgSeq):
    imgLabel.loc[imgLabel["image"]==int(val),"NeuralNet_Gray"]=Y_pred_case[i]
    if (Y_pred_case[i] in str(imgLabel.loc[imgLabel["image"]==int(val)]["label"])):
        correct = correct+1
print(imgLabel.head(10))
total = len(imgSeq)
genAcc = (correct/total) *100
print(correct, "Images classified correctly out of ",total, "images")
print("General Accuracy: ",genAcc)
imgLabel.to_csv("D:/athira/Courses/MS WPI/Fall'19/ML/Project_Distracted Driver Detection/Data/imgs/test 500/ImageLabels1.csv",index=False)
#41/120 correct

530
['c5' 'c5' 'c0' 'c0' 'c8' 'c2' 'c0' 'c0' 'c8' 'c8' 'c8' 'c0' 'c0' 'c2'
 'c7' 'c8' 'c8' 'c2' 'c5' 'c8' 'c6' 'c4' 'c6' 'c8' 'c4' 'c7' 'c5' 'c8'
 'c0' 'c7' 'c5' 'c4' 'c0' 'c0' 'c9' 'c5' 'c0' 'c9' 'c4' 'c3' 'c2' 'c8'
 'c7' 'c8' 'c6' 'c5' 'c7' 'c5' 'c8' 'c2' 'c8' 'c8' 'c5' 'c0' 'c4' 'c8'
 'c7' 'c7' 'c0' 'c4' 'c8' 'c7' 'c7' 'c8' 'c6' 'c7' 'c5' 'c9' 'c7' 'c6'
 'c1' 'c5' 'c8' 'c4' 'c0' 'c7' 'c5' 'c7' 'c8' 'c8' 'c7' 'c6' 'c8' 'c3'
 'c7' 'c7' 'c7' 'c5' 'c4' 'c2' 'c2' 'c7' 'c5' 'c3' 'c8' 'c6' 'c7' 'c8'
 'c1' 'c0' 'c4' 'c6' 'c7' 'c4' 'c5' 'c7' 'c4' 'c8' 'c0' 'c6' 'c4' 'c7'
 'c7' 'c7' 'c1' 'c5' 'c4' 'c4' 'c7' 'c8' 'c8' 'c4' 'c4' 'c8' 'c0' 'c4'
 'c7' 'c5' 'c7' 'c4' 'c2' 'c7' 'c7' 'c6' 'c6' 'c6' 'c7' 'c7' 'c2' 'c4'
 'c5' 'c4' 'c5' 'c8' 'c6' 'c7' 'c4' 'c8' 'c7' 'c2' 'c7' 'c5' 'c7' 'c6'
 'c7' 'c0' 'c8' 'c6' 'c7' 'c0' 'c6' 'c7' 'c5' 'c7' 'c8' 'c8' 'c7' 'c8'
 'c7' 'c1' 'c2' 'c1' 'c1' 'c2' 'c7' 'c8' 'c4' 'c7' 'c9' 'c5' 'c8' 'c2'
 'c5' 'c8' 'c0' 'c0' 'c4' 'c5' 'c6' 'c0' 'c2' 'c3' 'c8' 'c4' 'c2' 'c7'
 '